In [13]:
import numpy as np
from numpy import *
import random
import copy
from sklearn import metrics
from sklearn import mixture
import pandas as pd
import scipy

In [8]:
#EM算法称最大期望算法，是一类通过迭代进行极大似然估计的优化算法
data = pd.read_csv('gaussmix.csv')
print(data)

    0.064  -1.758  1.686
0  -0.191   0.463 -0.924
1  -0.529   0.782  0.585
2  -0.754   0.290  1.194
3   1.959  -0.512  2.737
4  -0.257  -1.371 -1.267
5   0.592   1.886  1.220
6  -0.001  -1.614 -1.228
7   0.002   0.884  0.582
8   0.440   0.565 -0.693
9   1.259   0.858 -2.105
10  0.207   0.220 -1.006
11  0.326   0.069 -1.499
12  1.732   2.232  0.147
13  0.035   0.798  0.896
14  0.617   0.303  1.448
15  0.527  -0.553  0.298
16  0.863   2.137  1.092
17  0.198  -1.794 -1.387
18  0.176   2.137  0.006
19  3.215   1.128  3.828
20 -1.404   0.658 -2.556
21  1.574  -0.516  1.227
22 -0.418  -0.021  0.228


In [9]:
num_iter = 1000
n,d = data.shape
mu1 = np.random.standard_normal(d)
mu2 = np.random.standard_normal(d)
sigma1 = np.identity(d)
sigma2 = np.identity(d)
pi = 0.5

In [10]:
for i in range(num_iter):
    #E-step
    norm1 = scipy.stats.multivariate_normal(mu1, sigma1)
    norm2 = scipy.stats.multivariate_normal(mu2, sigma2)
    tau1 = pi*norm1.pdf(data)
    tau2 = (1-pi)*norm2.pdf(data)
    gamma = tau1/(tau1+tau2)
    #M-step
    mu1 = np.dot(gamma, data)/sum(gamma)
    mu2 = np.dot((1-gamma), data)/sum((1-gamma))
    sigma1 = np.dot(gamma*(data - mu1).T, data - mu1)/np.sum(gamma)
    sigma2 = np.dot((1-gamma)*(data - mu1).T, data-mu1)/np.sum((1-gamma))
    pi = sum(gamma)/n

In [18]:
g = mixture.GaussianMixture(n_components=2, covariance_type='full', max_iter=1000)
g.fit(data)
print("weights" + "--"*10)
print(g.weights_)
print("means" + "--"*10)
print(g.means_, '\n')
# 新版本中没有g.covars_了
# print(g.covars_, '\n')

weights--------------------
[0.13043076 0.86956924]
means--------------------
[[ 2.24935407  0.03335026  2.59737527]
 [ 0.1710065   0.44644555 -0.24844314]] 



In [19]:
def generate_data():
    Miu1 = 2
    Miu2 = 4
    sigma1 = 1
    sigma2 = 2
    alpha1 = 0.4
    alpha2 = 0.6
    N = 5000
    N1 = int(alpha1 * N)
    X = mat(zeros((N,1)))
    for i in range(N1):
        temp = random.uniform(0,0.5)
        X[i] = temp * sigma1 + Miu1
    for i in range(N-N1):
        temp = random.uniform(0,0.5)
        X[i+N1] = temp * sigma2 + Miu2
    return X

In [24]:
def my_GMM(X):
    k = 2
    N = len(X)
    Miu = np.random.rand(k,1)
    Posterior = mat(zeros((N,k)))
    sigma = np.random.rand(k,1)
    sigma[0]=1
    #sigma[1]=2
    alpha = np.random.rand(k,1)
    alpha[0] = 0.1
    alpha[1] = 0.9
    dominator = 0
    numerator = 0
    # EPS是可调参数
    EPS = 0.1
    #先求后验概率
    print(sigma)
    for it in range(1000):
        for i in range(N):
            dominator = 0
            for j in range(k):
                dominator = dominator + np.exp(-1.0/(2.0*sigma[j]) * (X[i] - Miu[j])**2)
                #print -1.0/(2.0*sigma[j]),(X[i] - Miu[j])**2,-1.0/(2.0*sigma[j]) * (X[i] - Miu[j])**2,np.exp(-1.0/(2.0*sigma[j]) * (X[i] - Miu[j])**2)
                #return
            for j in range(k):
                numerator = np.exp(-1.0/(2.0*sigma[j]) * (X[i] - Miu[j])**2)
                Posterior[i,j] = numerator/dominator
        oldMiu = copy.deepcopy(Miu)
        oldalpha = copy.deepcopy(alpha)
        oldsigma = copy.deepcopy(sigma)
        #最大化
        for j in range(k):
            numerator = 0
            dominator = 0
            for i in range(N):
                numerator = numerator + Posterior[i,j] * X[i]
                dominator = dominator + Posterior[i,j]
            Miu[j] = numerator/dominator
            alpha[j] = dominator/N
            tmp = 0
            for i in range(N):
                tmp = tmp + Posterior[i,j]*(X[i] - Miu[j])**2
            #print tmp,Posterior[i,j],(X[i] - Miu[j])**2 
            sigma[j] = tmp/dominator
            print(tmp, dominator, sigma[j])
        if ((abs(Miu - oldMiu)).sum() < EPS) and ((abs(alpha - oldalpha)).sum() < EPS) and ((abs(sigma - oldsigma)).sum() < EPS):
            print(Miu,sigma,alpha,it)
            break

In [25]:
X = generate_data()
my_GMM(X)

[[1.        ]
 [0.94607058]]
[[3834.88219987]] 3153.9532979733885 [1.21589695]
[[2393.79034161]] 1846.0467020266221 [1.29671169]
[[2917.9062387]] 2473.335162154231 [1.17974559]
[[3294.53874452]] 2526.6648378457676 [1.3039081]
[[2846.47255315]] 2436.8829828786593 [1.16807929]
[[3347.1789527]] 2563.1170171213444 [1.30590173]
[[3.78116707]
 [3.43001309]] [[1.16807929]
 [1.30590173]] [[0.4873766]
 [0.5126234]] 2
